### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return 1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) +
                          (r_c ** 2) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c/l_s * np.log2(l_c/l_s), axis=1) - np.sum(
            r_c/r_s*np.log2(r_c/r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(
            r_c / (l_s + r_s), axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort, y_sort=self.__sort_samples(x,y)
        class_num=self.num_class
        cut = np.int(self.min_samples_split / 2 - 1)
        y_sort_slit = y_sort[cut:-cut] if cut != 0 else y_sort
        r_border_ids = np.where(y_sort_slit[:-1] != y_sort_slit[1:])[0] + (cut + 1)
        
        if len(r_border_ids) == 0:
            return np.inf, None
        
        eq_el_count = r_border_ids - np.append([cut], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_num))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     y_sort[r_border_ids - 1]] = 1
        
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y_sort[:cut], minlength=class_num)

        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sort, minlength=class_num) - l_class_count
    
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = y_sort.shape[0] - l_sizes

        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        indx = np.argmin(gs)
        
        left_el_id = l_sizes[indx][0]
        
        return gs[indx], (x_sort[left_el_id - 1] + x_sort[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and self.max_depth<=depth) or \
            y.size < self.min_samples_split or \
            self.sufficient_share <= np.bincount(y).argmax() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
        threshold = np.array([self.__find_threshold(x[:, i], y)
                              for i in self.get_feature_ids(x.shape[1])])
        best_id = threshold[:, 0].argmin()
        best_threshold = threshold[best_id, 1]
        
        if best_threshold == None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
        else:
            x_l, x_r, y_l, y_r =self.__div_samples(x, y,
                                                   best_id, best_threshold)
            if x_l.size == 0 or x_r.size == 0:
                self.tree[node_id] = (self.LEAF_TYPE,np.bincount(y).argmax(),
                                      np.bincount(y).astype(float) / y.size)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE,
                                      best_id, best_threshold)
                self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
            if self.G_function == self.__gini:
                g_r=(1 - np.sum(np.unique(y_r, return_counts=True)[1] ** 2 /
                                y_r.size ** 2))
                g=(1 - np.sum(np.unique(y, return_counts=True)[1]** 2 /
                              y.size ** 2))
                g_l=(1 - np.sum(np.unique(y_l, return_counts=True)[1] ** 2 /
                                y_l.size ** 2))
            if self.G_function == self.__entropy:
                g_r=-(np.sum((np.unique(y_r, return_counts=True)[1] / y_r.size) *
                             np.log2(np.unique(y_r, return_counts=True)[1] / y_r.size)))
                g=-(np.sum((np.unique(y, return_counts=True)[1] / y.size) *
                           np.log2(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=-(np.sum((np.unique(y_l, return_counts=True)[1] / y_l.size) *
                             np.log2(np.unique(y_l, return_counts=True)[1] / y_l.size)))  
            elif self.G_function == self.__misclass:
                g_r=(1 - np.max(np.sum(np.unique(y_r, return_counts=True)[1] / y.size)))
                g=(1 - np.max(np.sum(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=(1 - np.max(np.sum(np.unique(y_l, return_counts=True)[1] / y.size)))
            self.feature_importances_[best_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)
            / y.size
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 22.8 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8916666666666667

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8916666666666667

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB


In [10]:
for i in df.columns:
    if type(df[i][0]) not in [np.float64, np.int64]:
        df.drop(i, axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 187 entries, iid to amb5_3
dtypes: float64(174), int64(13)
memory usage: 12.0 MB


In [12]:
for i in df.columns:
    if df[i].isnull().sum()>500:
        df.drop(i,axis=1, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 82 columns):
iid         8378 non-null int64
id          8377 non-null float64
gender      8378 non-null int64
idg         8378 non-null int64
condtn      8378 non-null int64
wave        8378 non-null int64
round       8378 non-null int64
position    8378 non-null int64
order       8378 non-null int64
partner     8378 non-null int64
pid         8368 non-null float64
match       8378 non-null int64
int_corr    8220 non-null float64
samerace    8378 non-null int64
age_o       8274 non-null float64
race_o      8305 non-null float64
pf_o_att    8289 non-null float64
pf_o_sin    8289 non-null float64
pf_o_int    8289 non-null float64
pf_o_fun    8280 non-null float64
pf_o_amb    8271 non-null float64
pf_o_sha    8249 non-null float64
dec_o       8378 non-null int64
attr_o      8166 non-null float64
sinc_o      8091 non-null float64
intel_o     8072 non-null float64
fun_o       8018 non-null float64

In [14]:
df.drop('id', axis=1)

,iid,gender,idg,condtn,wave,round,position,order,partner,pid,...,intel3_1,amb3_1,dec,attr,sinc,intel,fun,like,prob,met
0,1,0,1,1,1,10,7,4,1,11.0,...,8.0,7.0,1,6.0,9.0,7.0,7.0,7.0,6.0,2.0
1,1,0,1,1,1,10,7,3,2,12.0,...,8.0,7.0,1,7.0,8.0,7.0,8.0,7.0,5.0,1.0
2,1,0,1,1,1,10,7,10,3,13.0,...,8.0,7.0,1,5.0,8.0,9.0,8.0,7.0,NaN,1.0
3,1,0,1,1,1,10,7,5,4,14.0,...,8.0,7.0,1,7.0,6.0,8.0,7.0,7.0,6.0,2.0
4,1,0,1,1,1,10,7,7,5,15.0,...,8.0,7.0,1,5.0,6.0,7.0,7.0,6.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,44,2,21,22,14,5,18,526.0,...,7.0,7.0,0,3.0,5.0,5.0,5.0,2.0,5.0,0.0
8374,552,1,44,2,21,22,13,4,19,527.0,...,7.0,7.0,0,4.0,6.0,8.0,4.0,4.0,4.0,0.0
8375,552,1,44,2,21,22,19,10,20,528.0,...,7.0,7.0,0,4.0,7.0,8.0,8.0,6.0,5.0,0.0
8376,552,1,44,2,21,22,3,16,21,529.0,...,7.0,7.0,0,4.0,6.0,5.0,4.0,5.0,5.0,0.0


In [15]:
y = df.loc[:, 'gender'].values
df=df.fillna(-8888)
X=df.drop('gender', axis=1).iloc[:,0:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)
myclf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [16]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

Wall time: 239 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time myclf.fit(X_train, y_train)

Wall time: 2.31 s


## Проверка качества работы на Speed Dating Data

In [18]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9896538231659138

In [19]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=myclf.predict(X_test), y_true=y_test, average='macro')

0.9761016469013575

## Задание 3

In [20]:
print(pd.Series(data=clf.feature_importances_,
          index=df.columns[1:]).sort_values(ascending=False)[:10])


attr1_1     0.237565
attr2_1     0.161761
pf_o_att    0.105738
shopping    0.052974
id          0.036227
amb2_1      0.033847
intel2_1    0.027798
amb1_1      0.023603
exphappy    0.023555
imprace     0.021547
dtype: float64


In [21]:
print(pd.Series(data=myclf.feature_importances_, 
        index=df.columns[1:]).sort_values(ascending=False)[:10])

attr1_1     722.435629
attr2_1     460.798461
pf_o_att    338.293168
id          172.872364
amb1_1      172.371334
shopping     96.829168
amb2_1       74.107186
intel2_1     73.837510
sports       53.913357
date         50.619175
dtype: float64


## Задание 4

In [22]:
parametrs = {
    "min_samples_split": [i for i in range(2, 20, 2)],
    "max_depth": [i for i in range(2, 20, 2)],
    "n_estimators": [i for i in range(1, 20,2)],
    "criterion": ["gini", "entropy"]
}
RSCV = RandomizedSearchCV(RandomForestClassifier(random_state=123), param_distributions=parametrs, n_iter=75)
RSCV.fit(X_train, y_train)
print(RSCV.best_params_)
f1_score(y_pred=RSCV.predict(X_test), y_true=y_test, average='macro')

C:\Users\Asia\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 15, 'min_samples_split': 4, 'max_depth': 12, 'criterion': 'entropy'}


0.9988063333653565

## Задание 5